Brandon Johnson

For this project I chose to use the pygame library to create a GUI. pygame is available for free down at https://www.pygame.org/news

In [1]:
import sys, pygame
from pygame.locals import *

To represent the maze for the game, I chose to give each tile a key:


Pac Man      = *

Blinky       = B

Pinky        = P

Inky         = I

Clyde        = C

Wall         = -

Dot          = .

Power Pellet = $

Empty Tile   = _

Warp Left    = L

Warp Right   = R

I have decided to represent the initial maze in a text file. After doing some quick analysis we have these totals:
868 tiles

568 walls

240 dots

52 empty tiles

4 power pellets

4 ghosts

2 warps

and

1 pac man

This gives us 240 + 52 + 4 + 2 + 1 + 1 = 300 walkable areas in the game. Note that only 1 ghost tile is accessible to Pac Man so I am only counting where Blinky starts at. Since each dot can turn into an empty tile that gives us * 240 = 72,000 + the possibility that each ghost and pac man can be in any of the 300 walkable areas as well. This quickly explodes to an unmanageable number of states. Because of this, we have to cut the number of states down drastically.

I am using the numpy loadtxt function to load a text file maze.txt that represents the maze. 

In [2]:
import numpy as np
largeMaze = np.loadtxt("maze.txt", dtype="str", delimiter=" ")
smallmaze = np.loadtxt("small-maze.txt", dtype="str", delimiter=" ")
size = 24

numCols = 16
numRows = 17
colSpacing = 4
displaywidth = size * numCols
displayheight = size * numRows
colwidth = 28
rowlength = 31

In [3]:
def printMaze(maze, colwidth):
    for i in range(len(maze)):
        if i != 0 and i % colwidth == 0:
            print()
        print(maze[i], end=" ")

In [4]:
printMaze(largeMaze, colwidth)

- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
- . . . . . . . . . . . . - - . . . . . . . . . . . . - 
- . - - - - . - - - - - . - - . - - - - - . - - - - . - 
- $ - - - - . - - - - - . - - . - - - - - . - - - - $ - 
- . - - - - . - - - - - . - - . - - - - - . - - - - . - 
- . . . . . . . . . . . . . . . . . . . . . . . . . . - 
- . - - - - . - - . - - - - - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - - - - - . - - . - - - - . - 
- . . . . . . - - . . . . - - . . . . - - . . . . . . - 
- - - - - - . - - - - - _ - - _ - - - - - . - - - - - - 
- - - - - - . - - - - - _ - - _ - - - - - . - - - - - - 
- - - - - - . - - _ _ _ _ B _ _ _ _ _ - - . - - - - - - 
- - - - - - . - - _ - - - - - - - - _ - - . - - - - - - 
- - - - - - . - - _ - _ _ _ _ _ _ - _ - - . - - - - - - 
L _ _ _ _ _ . _ _ _ - I _ P _ _ C - _ _ _ . _ _ _ _ _ R 
- - - - - - . - - _ - _ _ _ _ _ _ - _ - - . - - - - - - 
- - - - - - . - - _ - - - - - - - - _ - - . - - - - - - 
- - - - - - . - - _ _ _ _ _ _ _

In [5]:
printMaze(smallmaze, numCols)

- - - - - - - - - - - - - - - - 
- . . . . . . - - . . . . . . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . . . . . . B . . . . . . . - 
- . - - - _ - - - - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - - - - _ - - - . - 
- . . . . . . . * . . . . . . - 
- . - - - - - - - - - - - - . - 
- . - - - - - - - - - - - - . - 
- . . . . . . . . . . . . . . - 
- - - - - - - - - - - - - - - - 
  

In [6]:
import copy

class MovingObject:
    posX = 0
    posY = 0
    images = []
    anims = {}
    animPos = 0
    symbol = ""
    spacing = 0
    direction = ""
    spacesymbol = "_"
    size = 0
    prevX = -1
    prevY = -1
    def __init__(self, images, x, y, moveF, anims, symbol, spacing, size, direction):
        self.posX = x
        self.posY = y
        self.prevX = x
        self.prevY = y
        self.images = copy.copy(images)
        self.moveF = moveF
        self.anims = copy.deepcopy(anims)
        self.symbol = symbol
        self.spacing = spacing
        self.size = size
        self.direction = direction
        
    def getPosition(self):
        offsetX = -self.spacing
        offsetY = -self.spacing * 2
        if self.direction == 'left':
            offsetX -= self.spacing * self.animPos
        if self.direction == 'right':
            offsetX += self.spacing * self.animPos
        if self.direction == 'up':
            offsetY -= self.spacing * self.animPos
        if self.direction == 'down':
            offsetY += self.spacing * self.animPos
        return (self.posX*size+offsetX, self.posY*size+offsetY)
    
    def setPosition(self,x,y):
        self.posX = x
        self.posY = y
    
    def getImages(self):
        return self.images

    def getImage(self,index):
        return self.images[self.anims[self.direction][index]]
    
    def getAnimPos(self):
        return self.animPos
    
    def updatePos(self):
        if self.animPos == 0:
            if self.direction == 'left':
                self.prevX = self.posX
                self.posX -= 1
            if self.direction == 'right':
                self.prevX = self.posX
                self.posX += 1
            if self.direction == 'up':
                self.prevY = self.posY
                self.posY -= 1
            if self.direction == 'down':
                self.prevY = self.posY
                self.posY += 1
    
    def updateAnimPos(self, direct):
        if direct == self.direction:
            self.animPos = (self.animPos + 1) % len(self.anims[self.direction])
            self.updatePos()
        elif self.animPos == 0:
            self.direction = direct
        else:
            if (direct == 'left' and self.direction == 'right' or direct == 'right' and self.direction == 'left' or
                direct == 'up' and self.direction == 'down' or direct == 'down' and self.direction == 'up'):
                self.animPos = (len(self.anims) - self.animPos) % len(self.anims) 
                self.direction = direct
                self.updatePos()
        

    def getDir(self):
        return self.direction
    
    def setDir(self,direction):
        self.direction = direction
        
    def move(self,event,maze):
        self.moveF(self,event,maze)
        
    def setSpaceSymbol(self,maze):
        self.spacesymbol = maze[self.posX + self.posY * numCols]
        
    def getSpaceSymbol(self):
        return self.spacesymbol
    
    def getSymbol(self):
        return self.symbol

In [7]:
def noWall(direct,maze,sprite):
    if direct == 'left':
        return (maze[sprite.posX + sprite.posY * numCols - 1] != '-')
    if direct == 'right':
        return (maze[sprite.posX + sprite.posY * numCols + 1] != '-')
    if direct == 'up':
        return (maze[sprite.posX + (sprite.posY - 1) * numCols] != '-')
    if direct == 'down':
        return (maze[sprite.posX + (sprite.posY + 1) * numCols] != '-')

In [8]:
def getOppositeDir(direct):
    if direct == 'left':
        return 'right'
    elif direct == 'right':
        return 'left'
    elif direct == 'up':
        return 'down'
    elif direct == 'down':
        return 'up'

In [9]:
import copy
def takeActionF(startState, action, symbol):
    symbolIndex = np.where(startState==symbol)[0][0]
    tempState = copy.copy(startState)
    if (action == 'left'):
        tempState[symbolIndex],tempState[symbolIndex-1] = tempState[symbolIndex-1],tempState[symbolIndex]
    elif(action == 'right'):
        tempState[symbolIndex],tempState[symbolIndex+1] = tempState[symbolIndex+1],tempState[symbolIndex]
    elif(action == 'up'):
        tempState[bIndex],tempState[symbolIndex-3] = tempState[symbolIndex-3],tempState[symbolIndex]
    elif(action == 'down'):
        tempState[bIndex],tempState[symbolIndex+3] = tempState[symbolIndex+3],tempState[symbolIndex]
    
    return tempState, action[1]


def depthLimitedSearch(state, goalState, actions, takeActionF, depthLimit):
    # global nodes used to get the number of nodes encountered
    global num_nodes
    if state == goalState:
        return []
    if depthLimit == 0:
        return 'cutoff'
    cutoffOccurred = False
    for action in actions:
        num_nodes += 1 
        childState,_ = takeActionF(state, action)
        result = depthLimitedSearch(childState, goalState, actionsF, takeActionF, depthLimit-1)
        if result == 'cutoff':
            cutoffOccurred = True
        elif result != 'failure':
            result = [childState] + result[0:]
            return result
    if cutoffOccurred:
        return 'cutoff'
    else:
        return 'failure'


def iterativeDeepeningSearch(startState, goalState, actions, takeActionF, maxDepth):
    for depth in range(maxDepth):
        result = depthLimitedSearch(startState, goalState, actions, takeActionF, depth)
        if result == 'failure':
            return 'failure'
        if result != 'cutoff':
            result = [startState] + result[0:]
            return result
    return 'cutoff'

In [10]:
import random

def randomMove(sprite,events,maze):
    direct = sprite.getDir()
    if sprite.getAnimPos() == 0:
        dirs = ["left", "right", "up", "down"]
        options = []
        for d in dirs:
            if noWall(d,maze,sprite): 
                options.append(d)
        if len(options) > 0:
            direct = options[random.randint(0,len(options)-1)]
    sprite.updateAnimPos(direct)
    if sprite.getSymbol() != '*':
        sprite.setSpaceSymbol(maze)

In [11]:
def blinkyMove(sprite,events,maze):
    if sprite.getAnimPos() == 0:
        direct = sprite.getDir()
        dirs = ["left", "right", "up", "down"]
        # boolean to keep from abruptly turning
        changeDir = 0
        options = []
        for d in dirs:
            if noWall(d,maze,sprite): 
                options.append(d)
                if d != direct and d != getOppositeDir(direct):
                    changeDir = 1
        if changeDir:
            pacmanpos = np.where(maze=='*')[0][0]
            pacmanx = pacmanpos%numCols
            pacmany = math.floor(pacmanpos/numCols)
            best = getBestMove(options)
            direct = options[best]
    if sprite.getSymbol() != '*':
        sprite.setSpaceSymbol(maze)

In [12]:
def userMove(sprite,events,maze):
    for event in events:
        if event.type == KEYDOWN:
            direct = ''
            if event.key == K_LEFT:
                direct = 'left'
            elif event.key == K_RIGHT:
                direct = 'right'
            elif event.key == K_UP:
                direct = 'up'
            elif event.key == K_DOWN:
                direct = 'down'

            if direct == sprite.getDir():
                if(sprite.getAnimPos() == 0):
                    if noWall(direct, maze, sprite):
                        sprite.updateAnimPos(sprite.getDir())
                else:
                    sprite.updateAnimPos(sprite.getDir())
            else:
                sprite.updateAnimPos(direct)
        else:
            if sprite.getAnimPos() == 0:
                if noWall(sprite.getDir(), maze, sprite):
                    sprite.updateAnimPos(sprite.getDir())
            else:
                sprite.updateAnimPos(sprite.getDir())

    if sprite.getSymbol() != '*':
        sprite.setSpaceSymbol(maze)

In [13]:
import math

background = pygame.image.load("small-maze.png")
pacmanimages = []
pacmanimages.append("pacman-closed.png")
pacmanimages.append("pacman-open-down.png")
pacmanimages.append("pacman-open-left.png")
pacmanimages.append("pacman-open-right.png")
pacmanimages.append("pacman-open-up.png")
pacmanimages.append("pacman-wide-down.png")
pacmanimages.append("pacman-wide-left.png")
pacmanimages.append("pacman-wide-right.png")
pacmanimages.append("pacman-wide-up.png")
pacmananims = {}
pacmananims["down"] = [0,1,5,1]
pacmananims["left"] = [0,2,6,2]
pacmananims["right"] = [0,3,7,3]
pacmananims["up"] = [0,4,8,4]
pacmansymbol = '*'
pacmanpos = np.where(smallmaze==pacmansymbol)[0][0]
pacmanx = pacmanpos%numCols
pacmany = math.floor(pacmanpos/numCols)
pacman = MovingObject(pacmanimages, pacmanx, pacmany, randomMove, pacmananims, pacmansymbol, colSpacing, size, 'left')

ghosts = []
ghostsymbols = ['B','P','I','C']
ghostnames = ['blinky','pinky', 'inky', 'clyde']
for i in range(len(ghostsymbols)):
    if ghostsymbols[i] in smallmaze:
        ghostimages = [ghostnames[i]+"-down.png",ghostnames[i]+"-left.png",ghostnames[i]+"-right.png",
              ghostnames[i]+"-up.png", "frightened-blue.png", "frightened-white.png"]
        ghostanims = {}
        ghostanims["down"] = [0]
        ghostanims["left"] = [1]
        ghostanims["right"] = [2]
        ghostanims["up"] = [3]
        ghostanims["frightened"] = [4,5]
        ghostpos = np.where(smallmaze==ghostsymbols[i])[0][0]
        ghostx = ghostpos%numCols
        ghosty = math.floor(ghostpos/numCols)
        ghosts.append(MovingObject(ghostimages, ghostx, ghosty, randomMove,
               ghostanims, ghostsymbols[i], colSpacing, size, 'left'))

In [14]:
def showMaze(maze, screen, size):
    for i in range(len(maze)):
        x = i%numCols
        y = math.floor(i/numCols)
        if maze[i]=='.':    
            screen.blit(pygame.image.load("pellet.png"),(x*size+colSpacing*2,y*size+colSpacing*2)) 
        if maze[i]=='$':    
            screen.blit(pygame.image.load("power-pellet.png"),(x*size-colSpacing,y*size-colSpacing))

In [15]:
def updateMap(sprite, maze, ghosts, font, screen):
    maze[sprite.prevX + sprite.prevY * numCols] = sprite.getSpaceSymbol()
    if maze[sprite.posX + sprite.posY * numCols] == '.':
        maze[sprite.posX + sprite.posY * numCols] = sprite.getSymbol()
    for g in ghosts:
        maze[g.prevX + g.prevY * numCols] = g.getSpaceSymbol()
        if maze[g.posX + g.posY * numCols] == '.':
            maze[g.posX + g.posY * numCols] = g.getSymbol()
        if sprite.posX == g.posX and sprite.posY == g.posY:
            # screen.blit(font.render("Game Over!", False, (0,0,0)), math.floor(displaywidth/2), math.floor(displayheight/2))
            print ("Game Over!", displaywidth/2, displayheight/2)
            gameOver = 1

In [16]:
pygame.init()
pygame.font.init()
myfont = pygame.font.SysFont('Arial', 24)
screen = pygame.display.set_mode((displaywidth, displayheight))
pygame.display.set_caption('Pacman!')
clock = pygame.time.Clock()
gameOver = 0
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit
            gameOver = 1
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                pygame.quit()
                sys.exit
                gameOver = 1
    if not gameOver:
        screen.blit(background,(0,0))
        showMaze(smallmaze,screen,size)
        pacman.move(pygame.event.get(),smallmaze)
        [x.move(pygame.event.get(),smallmaze) for x in ghosts]
        updateMap(pacman, smallmaze, ghosts, myfont, screen)
        screen.blit(pygame.image.load(pacman.getImage(pacman.getAnimPos())),
            (pacman.getPosition()[0],pacman.getPosition()[1]))
        [screen.blit(pygame.image.load(x.getImage(x.getAnimPos())),(x.getPosition()[0],x.getPosition()[1])) for x in ghosts]
        pygame.display.flip()
        clock.tick(60)
        printMaze(smallmaze,numCols)
        print()

- - - - - - - - - - - - - - - - 
- . . . . . . - - . . . . . . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . . . . . B . . . . . . . . - 
- . - - - _ - - - - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - - - - _ - - - . - 
- . . . . . . . _ . . . . . . - 
- . - - - - - - - - - - - - . - 
- . - - - - - - - - - - - - . - 
- . . . . . . . . . . . . . . - 
- - - - - - - - - - - - - - - - 
  
- - - - - - - - - - - - - - - - 
- . . . . . . - - . . . . . . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . . . . . B B . . . . . . . - 
- . - - - _ - - - - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - - - - _ - - - . - 
- . . . . . . . _ . . . . . . - 
- . - -

- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - - - - _ - - - . - 
- . . . . _ _ _ _ . . . . . . - 
- . - - - - - - - - - - - - . - 
- . - - - - - - - - - - - - . - 
- . . . . . . . . . . . . . . - 
- - - - - - - - - - - - - - - - 
  
- - - - - - - - - - - - - - - - 
- . . . . . . - - . . . . . . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . - - - - B B - . - - - - . - 
- . . . . . B B B . . . . . . - 
- . - - - _ - - - - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - - - - _ - - - . - 
- . . . . _ _ _ _ . . . . . . - 
- . - - - - - - - - - - - - . - 
- . - - - - - - - - - - - - . - 
- . . . . . . . . . . . . . . - 
- - - - - - - - - - - - - - - - 
  
- - - - - - - - - - - - - - - - 
- . . . . . . - - . . . . . . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . - - - - B B . . - - - - . - 
- . 

- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - - - - _ - - - . - 
- . . . . _ _ _ _ _ _ * . . . - 
- . - - - - - - - - - - - - . - 
- . - - - - - - - - - - - - . - 
- . . . . . . . . . . . . . . - 
- - - - - - - - - - - - - - - - 
  
- - - - - - - - - - - - - - - - 
- . . . . . . - - . . . . . . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . - - - - B B B B - - - - . - 
- . . . . . B B B B . . . . . - 
- . - - - _ - - - - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - - - - _ - - - . - 
- . . . . _ _ _ _ _ _ * . . . - 
- . - - - - - - - - - - - - . - 
- . - - - - - - - - - - - - . - 
- . . . . . . . . . . . . . . - 
- - - - - - - - - - - - - - - - 
  
- - - - - - - - - - - - - - - - 
- . . . . . . - - . . . . . . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . 

- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - - - - _ _ - - . - 
- . . . . _ _ _ _ _ _ _ . . . - 
- . - - - - - - - - - - - - . - 
- . - - - - - - - - - - - - . - 
- . . . . . . . . . . . . . . - 
- - - - - - - - - - - - - - - - 
  
- - - - - - - - - - - - - - - - 
- . . . . . . - - . . . . . . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . - - - - B B B B - - - - . - 
- . . . . . B B B B . . . . . - 
- . - - - _ - - - - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - _ _ - _ - - - . - 
- . - - - _ - _ _ - _ _ - - . - 
- . - - - _ - - - - _ _ - - . - 
- . . . . _ _ _ _ _ _ _ . . . - 
- . - - - - - - - - - - - - . - 
- . - - - - - - - - - - - - . - 
- . . . . . . . . . . . . . . - 
- - - - - - - - - - - - - - - - 
  
- - - - - - - - - - - - - - - - 
- . . . . . . - - . . . . . . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . 

- . - - - _ - _ _ - _ _ - - . - 
- . - - - _ - _ _ - _ _ - - . - 
- . - - - _ - _ _ - _ _ - - . - 
- . - - - _ - - - - _ _ - - . - 
- . . . . _ _ _ _ _ _ _ . . . - 
- . - - - - - - - - - - - - . - 
- . - - - - - - - - - - - - . - 
- . . . . . . . . . . . . . . - 
- - - - - - - - - - - - - - - - 
  
- - - - - - - - - - - - - - - - 
- . . . . . . - - . . . . . . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . - - - - B B B B - - - - . - 
- . . . . . B B B B * . . . . - 
- . - - - _ - - - - _ _ - - . - 
- . - - - _ - _ _ - _ _ - - . - 
- . - - - _ - _ _ - _ _ - - . - 
- . - - - _ - _ _ - _ _ - - . - 
- . - - - _ - - - - _ _ - - . - 
- . . . . _ _ _ _ _ _ _ . . . - 
- . - - - - - - - - - - - - . - 
- . - - - - - - - - - - - - . - 
- . . . . . . . . . . . . . . - 
- - - - - - - - - - - - - - - - 
  
- - - - - - - - - - - - - - - - 
- . . . . . . - - . . . . . . - 
- . - - - - . - - . - - - - . - 
- . - - - - . - - . - - - - . - 
- . 

...

- . . . . _ _ _ _ _ _ _ . . . - 
- . - - - - - - - - - - - - . - 
- . - - - - - - - - - - - - . - 
- . . . . . . . . . . . . . . - 
- - - - - - - - - - - - - - - - 
  
- - - - - - - - - - - - - - - - 
- . . . . . B B - . . . . . . - 
- . - - - - B B - . - - - - . - 
- . - - - - B B - . - - - - . - 
- . - - - B B B - . - - - - . - 
- B B B B B B B _ _ _ - - - . - 
- B B . _ B B B _ _ _ _ . . . - 
- B B - _ _ - - _ _ _ _ - - . - 
- B B - _ _ - _ _ - _ _ - - . - 
- B B - _ _ - _ _ - _ _ - - . - 
- . B - _ _ - _ _ - _ _ - - . - 
- B B - _ _ _ _ _ _ _ _ - - . - 
- . . . . _ _ _ _ _ _ _ . . . - 
- . - - - - - - - - - - - - . - 
- . - - - - - - - - - - - - . - 
- . . . . . . . . . . . . . . - 
- - - - - - - - - - - - - - - - 
  
- - - - - - - - - - - - - - - - 
- . . . . . B B - . . . . . . - 
- . - - - - B B - . - - - - . - 
- . - - - - B B - . - - - - . - 
- . - - - B B B - . - - - - . - 
- B B B B B B B _ _ _ - - - . - 
- B B . _ B B B _ _ _ _ . . . - 
- B B - _ _ - - _ _ _ _ - - . - 
- B 

- . . . . . B B - . . . . . . - 
- . - - - - B B - . - - - - . - 
- . - - - - B B - . - - - - . - 
- . - - - B B B - . - - - - . - 
- B B B B B B B _ _ _ - - - . - 
- B B . _ B B B _ _ _ _ . . . - 
- B B - _ _ - - _ _ _ _ - - . - 
- B B - _ _ - _ _ - _ _ - - . - 
- B B - _ _ - _ _ - _ _ - - . - 
- . B - _ _ - _ _ - _ _ - - . - 
- B B - _ _ _ _ _ _ _ _ - - . - 
- . . . . _ _ _ _ _ _ _ . . . - 
- . - - - - - - - - - - - - . - 
- . - - - - - - - - - - - - . - 
- . . . . . . . . . . . . . . - 
- - - - - - - - - - - - - - - - 
  
- - - - - - - - - - - - - - - - 
- . . . . . B B - . . . . . . - 
- . - - - - B B - . - - - - . - 
- . - - - - B B - . - - - - . - 
- . - - - B B B - . - - - - . - 
- B B B B B B B _ _ _ - - - . - 
- B B . _ B B B _ _ _ _ . . . - 
- B B - _ _ - - _ _ _ _ - - . - 
- B B - _ _ - _ _ - _ _ - - . - 
- B B - _ _ - _ _ - _ _ - - . - 
- . B - _ _ - _ _ - _ _ - - . - 
- B B - _ _ _ _ _ _ _ _ - - . - 
- . . . . _ _ _ _ _ _ _ . . . - 
- . - - - - - - - - - - - - . - 
- . - -

- B B - _ _ _ _ _ _ _ . . . . - 
- . . . . _ _ _ _ _ _ _ B B B - 
- . - - - - - - - - - - - - . - 
- . - - - - - - - - - - - - . - 
- . . . . . . . . . . . . . . - 
- - - - - - - - - - - - - - - - 
  
- - - - - - - - - - - - - - - - 
- . . . . . B B - . . . . . . - 
- . - - - - B B - . - - - - . - 
- . - - - - B B - . - - - - . - 
- . - - - B B B - . - - - - . - 
- B B B B B B B _ _ _ - - - . - 
- B B . _ B B B _ _ _ _ . . . - 
- B B - _ _ - - _ _ _ _ - - . - 
- B B - _ _ - _ _ - _ _ - - . - 
- B B - _ _ - _ _ - _ _ - - . - 
- . B - _ _ - _ _ - _ _ - - . - 
- B B - _ _ _ _ _ _ _ . . B . - 
- . . . . _ _ _ _ _ _ _ B B B - 
- . - - - - - - - - - - - - . - 
- . - - - - - - - - - - - - . - 
- . . . . . . . . . . . . . . - 
- - - - - - - - - - - - - - - - 
  
- - - - - - - - - - - - - - - - 
- . . . . . B B - . . . . . . - 
- . - - - - B B - . - - - - . - 
- . - - - - B B - . - - - - . - 
- . - - - B B B - . - - - - . - 
- B B B B B B B _ _ _ - - - . - 
- B B . _ B B B _ _ _ _ . . . - 
- B 

- . - - - - - - - - - - - - . - 
- . . . . . . . . . . . . . . - 
- - - - - - - - - - - - - - - - 
  
- - - - - - - - - - - - - - - - 
- . . . . . B B - . . . . . . - 
- . - - - - B B - . - - - - . - 
- . - - - - B B - . - - - - . - 
- . - - - B B B - . - - - - . - 
- B B B B B B B _ _ _ - - - . - 
- B B . _ B B B _ _ _ _ * . . - 
- B B - _ _ - - _ _ _ _ _ - . - 
- B B - _ _ - _ _ - _ _ - - . - 
- B B - _ _ - _ _ - _ _ - - . - 
- . B - _ _ - _ _ - _ _ - B B - 
- B B - _ _ _ _ _ _ _ . . B B - 
- . . . . _ _ _ _ _ _ _ B B B - 
- . - - - - - - - - - - - - . - 
- . - - - - - - - - - - - - . - 
- . . . . . . . . . . . . . . - 
- - - - - - - - - - - - - - - - 
  
- - - - - - - - - - - - - - - - 
- . . . . . B B - . . . . . . - 
- . - - - - B B - . - - - - . - 
- . - - - - B B - . - - - - . - 
- . - - - B B B - . - - - - . - 
- B B B B B B B _ _ _ - - - . - 
- B B . _ B B B _ _ _ _ * . . - 
- B B - _ _ - - _ _ _ _ _ - . - 
- B B - _ _ - _ _ - _ _ - - . - 
- B B - _ _ - _ _ - _ _ - - . - 
- . 

error: video system not initialized